In [1]:
import matplotlib
matplotlib.use('TKAgg')
%load_ext autoreload
%autoreload 2
from lab.mplc.discrete_scan_result import DiscreetScanResult
from misc.misc import run_in_thread_simple, run_in_thread
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_modes2 import gen_input_spots_array, gen_output_modes_Unitary
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.mask_utils import remove_input_modes
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_QKD_masks, create_WFM_unitary_masks
from pianoq.misc.misc import detect_gaussian_spots_subpixel
import time 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

d_no = 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cant use py_thorlabs_ctrl.kinesis
could not import ThorlabsRotatingServoMotor
cant use py_thorlabs_ctrl.kinesis


NameError: name 'KCubeDCServo' is not defined

# Generate masks WFM

In [ ]:
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_diffuser_masks

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path =    fr'{dir_path}\{timestamp}_diffuser_d{d_no}d.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_diffuser_d{d_no}d.mplcsim'
mplc_sim = create_WFM_diffuser_masks(N_iterations=15, out_path=masks_path)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

# HW

In [ ]:
# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

# Motors
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=10, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)
print('Got Time tagger')

# Two photons scan

In [ ]:
masks = None 
mplc.set_masks(masks)

# Put idler in OK place 
mxi.move_absolute(9.02)
myi.move_absolute(2.4)

# scan signal
start_x = 11
end_x = 12
start_y = 9
end_y = 10
integration_time = 10
tt.set_integration_time(integration_time)

x_pixels = 20
y_pixels = 20
pixel_size_x = 0.05
pixel_size_y = 0.05


timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = f'{dir_path}\\{timestamp}_signal_speckles_d{d_no}d.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, 
                        is_timetagger=True, coin_window=2e-9, saveto_path=path)
single1s, single2s, coincidences = scanner.scan()